### Inputs

In [36]:
from datasets import load_dataset
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import cosine_similarity

### Descargas

In [ ]:
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\scard\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\scard\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\scard\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Cargar Datos

In [12]:
dataset = load_dataset("wilfredomartel/small-spanish-legal-dataset", split="train")
df = pd.DataFrame(dataset)
for index, row in df.head().iterrows():
    print(f"<----------- Fila {index} ----------->")
    print(f"Query: {row['query']}")
    print(f"Positive: {row['pos'][0][:150]}...\n")

<----------- Fila 0 ----------->
Query: ¿Cuál fue la razón principal por la que la Corte Constitucional inadmitió a trámite la acción extraordinaria de protección N° 2850-19-EP, presentada por Javier Jefferson Delgado Guano y otros contra la sentencia de la Sala Única Multicompetente de la Corte Provincial de Esmeraldas?
Positive: La Corte Constitucional inadmitió a trámite la acción extraordinaria de protección N° 2850-19-EP debido a que los accionantes no presentaron un argume...

<----------- Fila 1 ----------->
Query: ¿Por qué la Sala de Admisión de la Corte Constitucional inadmitió a trámite la acción extraordinaria de protección N°. 3090-19-EP presentada por José Antonio Guzñay contra la sentencia de la Corte Provincial de Zamora?
Positive: La Sala de Admisión de la Corte Constitucional inadmitió a trámite la acción extraordinaria de protección N°. 3090-19-EP, presentada por el señor José...

<----------- Fila 2 ----------->
Query: ¿Cuál fue la controversia principal en el caso N

### Preprocesamiento

In [21]:
stemmer = nltk.SnowballStemmer("spanish")
stop_words_es = set(stopwords.words("spanish"))

def preprocesar_texto(texto):
    texto_minusculas = texto.lower() 
    tokens = word_tokenize(texto_minusculas)
    tokens_limpios = []
    for palabra in tokens:
        if palabra.isalpha(): 
            if palabra not in stop_words_es: 
                palabra_raiz = stemmer.stem(palabra) 
                tokens_limpios.append(palabra_raiz)
    return " ".join(tokens_limpios)

corpus_original = df['pos'].explode().dropna().unique()
corpus_limpio = [preprocesar_texto(texto) for texto in corpus_original]

print("<---- Prueba de Preprocesamiento ---->")
print(f"Texto Original:\n{corpus_original[0][:200]}...\n")
print(f"Texto Procesado:\n{corpus_limpio[0][:200]}...")

<---- Prueba de Preprocesamiento ---->
Texto Original:
La Corte Constitucional inadmitió a trámite la acción extraordinaria de protección N° 2850-19-EP debido a que los accionantes no presentaron un argumento claro sobre los derechos supuestamente violado...

Texto Procesado:
cort constitucional inadmit tramit accion extraordinari proteccion deb accion present argument clar derech supuest viol relacion direct inmediat accion u omision sal unic multicompetent cort provincia...


### TF-IFD

In [ ]:
vectorizer = TfidfVectorizer()
matriz_tfidf = vectorizer.fit_transform(corpus_limpio)
print("Dimensiones TF-IDF:", matriz_tfidf.shape) #total de documentos #palabras únicas

Dimensiones TF-IDF: (22458, 11626)


### Búsqueda

In [ ]:
def buscar_tfidf(pregunta_usuario, top_n=5):
    print(f"<--- Buscando: '{pregunta_usuario}' --->")
    pregunta_procesada = preprocesar_texto(pregunta_usuario)
    pregunta_vector = vectorizer.transform([pregunta_procesada])
    similitudes = cosine_similarity(pregunta_vector, matriz_tfidf).flatten()
    indices_top = similitudes.argsort()[-top_n:][::-1]
    print("Resultados encontrados:\n")
    for i in indices_top:
        print(f"  Puntaje de Similitud: {similitudes[i]:.4f}")
        print(f"  Texto: {corpus_original[i][:250]}...\n")

pregunta_de_prueba = df.iloc[20]['query']

In [30]:
buscar_tfidf(pregunta_de_prueba)

<--- Buscando: '¿Cuál fue la razón principal por la que la Corte Constitucional inadmitió a trámite la acción extraordinaria de protección N. 2511-18-EP presentada por Juan Ricardo Cedillo Rizo?' --->
Resultados encontrados:

  Puntaje de Similitud: 0.6206
  Texto: El recurso de apelación interpuesto por Juan Ricardo Cedillo Rizo contra la sentencia condenatoria fue desestimado por la Sala de lo Penal de la Corte Provincial de Justicia de Manabí el 09 de mayo de 2018. Esta decisión mantuvo incólume la sentencia...

  Puntaje de Similitud: 0.6184
  Texto: La Sala de lo Penal de la Corte Provincial de Justicia de Manabí, mediante fallo dictado el 09 de mayo del 2018, desechó el recurso de apelación presentado por el señor Juan Ricardo Cedillo Rizo. En dicha resolución, la Sala confirmó en todas sus par...

  Puntaje de Similitud: 0.4594
  Texto: El señor Juan Ricardo Cedillo Rizo fue procesado y posteriormente condenado por el delito de asociación ilícita, conforme a lo establecido en lo

In [31]:
buscar_tfidf("¿Qué pasa con las obligaciones del vendedor?")

<--- Buscando: '¿Qué pasa con las obligaciones del vendedor?' --->
Resultados encontrados:

  Puntaje de Similitud: 0.3872
  Texto: Tras el contrato de compraventa, se inició un litigio donde Austrocorp S.A. demandó una reducción del precio por supuesta menor cabida del terreno vendido. En contrapartida, los vendedores reconvinieron exigiendo el pago de los $150.000,00 adeudados ...

  Puntaje de Similitud: 0.1940
  Texto: La decisión de la Sala de Admisión de la Corte Constitucional de admitir a trámite la acción extraordinaria de protección No. 1871-10-EP tiene una implicación fundamental: significa que la Corte ha considerado que la demanda cumple con los presupuest...

  Puntaje de Similitud: 0.1751
  Texto: La accionante, Ana Lucía Romero Medina de Santos, fundamentó su acción extraordinaria de protección en la presunta vulneración de dos derechos constitucionales fundamentales por parte del Juez Primero de Inquilinato de Guayaquil en el proceso de desa...

  Puntaje de Similitud: